In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import SnowballStemmer

In [ ]:
stops = set(stopwords.words("english"))
print(stops)

alternative_stops = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This']

In [ ]:
# Parameters:
# hyper_cleaning: If true, do a second round, more through data cleaning
# remove_punctuation: If true, remove all punctuations in the text
# remove_stopwords: If true, remove all stopwords (the list of stop words is shown above)
# use_alternative_stopwords: If true, remove all stopwords in a smaller set of stopwords
# stem_words: If true, shorten words to their stems

def data_cleaning(input_cvs_file_name_array, hyper_cleaning = False, remove_punctuation = True, remove_stopwords = True, use_alternative_stopwords = True, stem_words = False):
    for input_cvs_file_name in input_cvs_file_name_array:
        print("Cleaning " + input_cvs_file_name + "...")
        data = pd.read_csv(input_cvs_file_name)
        contents = data['cleanedContent']

        contents_cleaned = []

        for content in contents:
            cleaned_content = text_to_wordlist(str(content), hyper_cleaning, remove_punctuation, remove_stopwords, use_alternative_stopwords, stem_words)
            contents_cleaned.append(cleaned_content)

        data['cleanedContent'] = contents_cleaned

        output_cvs_file_name = input_cvs_file_name.split(".")[0] + "(cleaned)"

        if hyper_cleaning:
            output_cvs_file_name = output_cvs_file_name + "(hyper_cleaned)"
        if remove_punctuation:
            output_cvs_file_name = output_cvs_file_name + "(punctuation_removed)"
        if remove_stopwords:
            output_cvs_file_name = output_cvs_file_name + "(stopwords_removed)"
        if use_alternative_stopwords:
            output_cvs_file_name = output_cvs_file_name + "(alternative_stopwords_used)"
        if stem_words:
            output_cvs_file_name = output_cvs_file_name + "(words_shortened)"

        data.to_csv(output_cvs_file_name + '.csv', index=False)

In [ ]:
def text_to_wordlist(text, hyper_cleaning, remove_punctuation, remove_stopwords, use_alternative_stopwords, stem_words):
    
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        if use_alternative_stopwords:
            stops = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This']
        else:
            stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]

    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    if hyper_cleaning:
        # Clean the text (second round)
        text = re.sub(r"[^A-Za-z0-9]", " ", text)
        text = re.sub(r"what's", "", text)
        text = re.sub(r"What's", "", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"can't", "cannot ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"I'm", "I am", text)
        text = re.sub(r" m ", " am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r"60k", " 60000 ", text)
        text = re.sub(r" e g ", " eg ", text)
        text = re.sub(r" b g ", " bg ", text)
        text = re.sub(r"\0s", "0", text)
        text = re.sub(r" 9 11 ", "911", text)
        text = re.sub(r"e-mail", "email", text)
        text = re.sub(r"\s{2,}", " ", text)
        text = re.sub(r"quikly", "quickly", text)
        text = re.sub(r" usa ", " America ", text)
        text = re.sub(r" USA ", " America ", text)
        text = re.sub(r" u s ", " America ", text)
        text = re.sub(r" uk ", " England ", text)
        text = re.sub(r" UK ", " England ", text)
        text = re.sub(r"india", "India", text)
        text = re.sub(r"switzerland", "Switzerland", text)
        text = re.sub(r"china", "China", text)
        text = re.sub(r"chinese", "Chinese", text) 
        text = re.sub(r"imrovement", "improvement", text)
        text = re.sub(r"intially", "initially", text)
        text = re.sub(r"quora", "Quora", text)
        text = re.sub(r" dms ", "direct messages ", text)  
        text = re.sub(r"demonitization", "demonetization", text) 
        text = re.sub(r"actived", "active", text)
        text = re.sub(r"kms", " kilometers ", text)
        text = re.sub(r"KMs", " kilometers ", text)
        text = re.sub(r" cs ", " computer science ", text) 
        text = re.sub(r" upvotes ", " up votes ", text)
        text = re.sub(r" iPhone ", " phone ", text)
        text = re.sub(r"\0rs ", " rs ", text) 
        text = re.sub(r"calender", "calendar", text)
        text = re.sub(r"ios", "operating system", text)
        text = re.sub(r"gps", "GPS", text)
        text = re.sub(r"gst", "GST", text)
        text = re.sub(r"programing", "programming", text)
        text = re.sub(r"bestfriend", "best friend", text)
        text = re.sub(r"dna", "DNA", text)
        text = re.sub(r"III", "3", text) 
        text = re.sub(r"the US", "America", text)
        text = re.sub(r"Astrology", "astrology", text)
        text = re.sub(r"Method", "method", text)
        text = re.sub(r"Find", "find", text) 
        text = re.sub(r"banglore", "Banglore", text)
        text = re.sub(r" J K ", " JK ", text)

    if remove_punctuation:
        # Remove punctuation from text
        text = ''.join([c for c in text if c not in punctuation])

    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)

    # Return a list of words
    return(text)

In [ ]:
# load data
input_cvs_file_name_array = ['comp401-Fall-2012_labeled (new).csv', 'comp401-Fall-2013_labeled (new).csv', 'comp401-Fall-2015_labeled (new).csv', 'comp401-Fall-2016_labeled (new).csv', 'comp401-Fall-2017_labeled (new).csv', 'comp401-Fall-2018_labeled (new).csv', 'comp411-Fall-2019_labeled (new).csv', 'comp411-Fall-2020_labeled (new).csv', 'comp411-Spring-2019_labeled (new).csv', 'comp411-Spring-2020A_labeled (new).csv', 'comp411-Spring-2020B_labeled (new).csv', 'comp426-Fall-2019_labeled (new).csv', 'comp426-Fall-2020_labeled (new).csv']
data_cleaning(input_cvs_file_name_array)